# Introduction

In this project we are focused on building the advanced regression model using regularisation to understand the variables which are significant in predicting the price of a house & how well those variables describe the price of a house which will then be used by the management to manipulate strategy as per price variation with variables.

---

# Problem Statement

A US-based housing company named Surprise Housing has decided to enter the Australian market. The company uses data analytics to purchase houses at a price below their actual values and flip them on at a higher price. For the same purpose, the company has collected a data set from the sale of houses in Australia. The data is provided in the CSV file below.

 

The company is looking at prospective properties to buy to enter the market. You are required to build a regression model using regularisation in order to predict the actual value of the prospective properties and decide whether to invest in them or not.

 

<b>The company wants to know:</b>
1. Which variables are significant in predicting the price of a house, and
2. How well those variables describe the price of a house.

 

Also, determine the optimal value of lambda for ridge and lasso regression.

---

# Business Objective

Required to model the price of houses with the available independent variables. This model will then be used by the management to understand how exactly the prices vary with the variables. They can accordingly manipulate the strategy of the firm and concentrate on areas that will yield high returns. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

---

# Solution Approach

1. Reading, Understanding & Cleaning the Data
2. Visualising the Data
3. Data Preparation for modelling
4. Splitting the Data into Training and Testing Sets
5. Model Building and Evaluation
6. Ridge Regression
7. Lasso Regression
8. Model Comparison
9. Conclusion

----

# <u>Importing required libraries</u>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

# <u>Step 1: Reading, Understanding & Cleaning the Data</u>

In [2]:
# Loading data from csv

housesale_data = pd.read_csv("train.csv")

## Data Understanding

In [3]:
# Checking rows and columns in dataset
housesale_data.shape

(1460, 81)

In [4]:
# Checking general information about dataset, mostly to see various datatypes available in dataset
housesale_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
# Quick data view & also checking if any extra header rows present
housesale_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
# Checking if any extra rows present at bottom eg. total rows
housesale_data.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [7]:
# Checking the columns via describe

housesale_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## Data Cleaning

In [8]:
# Checking if any blank rows in dataset, if yes we would have to remove it
blank_rows = housesale_data[housesale_data.isnull().all(axis=1)]

# Print the blank rows
print(blank_rows)

Empty DataFrame
Columns: [Id, MSSubClass, MSZoning, LotFrontage, LotArea, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, YearBuilt, YearRemodAdd, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, Heating, HeatingQC, CentralAir, Electrical, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, KitchenQual, TotRmsAbvGrd, Functional, Fireplaces, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PavedDrive, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, PoolQC, Fence, MiscFeature, MiscVal, MoSold, YrSold, SaleType, SaleCondition, SalePrice]
Index: []

[0 rows x 81 columns]


- There are no blank rows in data set as well

In [9]:
# Checking duplicate rows in data

print(housesale_data.duplicated().sum())

0


- There are no duplicate rows in data set

In [10]:
# Get columns with all unique values
unique_value_columns = housesale_data.columns[housesale_data.nunique() == len(housesale_data)]

unique_value_columns

Index(['Id'], dtype='object')

In [11]:
# Lets drop the ID column as it cannot be used for analysis
housesale_data = housesale_data.drop('Id',axis='columns')

In [12]:
# Checking shape of housesale_data after to verify if column count is as expected
housesale_data.shape

(1460, 80)

#### Handling NULL Values in Columns

In [13]:
# Checking % of null present in all columns to see if any columns we can drop

null_percent_per_column = ((housesale_data.isnull().sum() / len(housesale_data)) * 100)

null_percent_per_column.round(2).sort_values(ascending=False)

PoolQC         99.52
MiscFeature    96.30
Alley          93.77
Fence          80.75
FireplaceQu    47.26
               ...  
Heating         0.00
HeatingQC       0.00
MSZoning        0.00
1stFlrSF        0.00
SalePrice       0.00
Length: 80, dtype: float64

In [14]:
# Dropping columns with more than 80% NULL Values in it

cols_to_drop = ['PoolQC', 'MiscFeature', 'Alley', 'Fence']
housesale_data.drop(cols_to_drop, axis = 1, inplace=True)

# Checking shape of housesale_data after to verify if column count is as expected
housesale_data.shape

(1460, 76)

In [15]:
# Check remaining columns and NULL values in it
housesale_data.isna().sum().sort_values(ascending = False).head(20)

FireplaceQu     690
LotFrontage     259
GarageType       81
GarageYrBlt      81
GarageFinish     81
GarageQual       81
GarageCond       81
BsmtExposure     38
BsmtFinType2     38
BsmtQual         37
BsmtCond         37
BsmtFinType1     37
MasVnrType        8
MasVnrArea        8
Electrical        1
Fireplaces        0
KitchenQual       0
KitchenAbvGr      0
BedroomAbvGr      0
HalfBath          0
dtype: int64

In [16]:
# Accoridng to the data defination - NA value have a real meaning in few variables
# e.g. FireplaceQu NA means 'No Fireplace', GarageType NA means 'No Garage'
# So replacing 'NA' values with 'None' value in such vaiables makes sense so python does not consider them as NULL

columns_having_meaningful_NA_value = ['FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 
                                    'BsmtExposure', 'BsmtFinType2', 'BsmtQual', 'BsmtCond', 'BsmtFinType1']

for col in columns_having_meaningful_NA_value:
    housesale_data[col] = housesale_data[col].fillna('None')
    

In [17]:
# Check remaining columns and NULL values in it
housesale_data.isna().sum().sort_values(ascending = False).head(20)

LotFrontage     259
GarageYrBlt      81
MasVnrType        8
MasVnrArea        8
Electrical        1
KitchenAbvGr      0
FireplaceQu       0
Fireplaces        0
Functional        0
TotRmsAbvGrd      0
KitchenQual       0
MSSubClass        0
GarageType        0
HalfBath          0
FullBath          0
BsmtHalfBath      0
BsmtFullBath      0
GrLivArea         0
LowQualFinSF      0
2ndFlrSF          0
dtype: int64

In [18]:
# Checking values in LotFrontage, GarageYrBlt, MasVnrType, MasVnrArea, Electrical column
housesale_data[['GarageYrBlt', 'LotFrontage', 'MasVnrType', 'MasVnrArea', 'Electrical']]

,GarageYrBlt,LotFrontage,MasVnrType,MasVnrArea,Electrical
0,2003.0,65.0,BrkFace,196.0,SBrkr
1,1976.0,80.0,None,0.0,SBrkr
2,2001.0,68.0,BrkFace,162.0,SBrkr
3,1998.0,60.0,None,0.0,SBrkr
4,2000.0,84.0,BrkFace,350.0,SBrkr
...,...,...,...,...,...
1455,1999.0,62.0,None,0.0,SBrkr
1456,1978.0,85.0,Stone,119.0,SBrkr
1457,1941.0,66.0,None,0.0,SBrkr
1458,1950.0,68.0,None,0.0,FuseA


In [19]:
# Filling LotFrontage Continuous variables with median value
housesale_data['LotFrontage']= housesale_data['LotFrontage'].fillna(housesale_data['LotFrontage'].median())

# Filling GarageYrBlt Continuous variables with median value
housesale_data['GarageYrBlt']= housesale_data['GarageYrBlt'].fillna(0.0)

# Filling MasVnrType with None (No Masonry veneer)
housesale_data['MasVnrType']= housesale_data['MasVnrType'].fillna('None')

# Filling MasVnrArea with 0 (No Masonry veneer - zero area)
housesale_data['MasVnrArea']= housesale_data['MasVnrArea'].fillna(0)

# Filling Electrical with mode value (SBrkr)
housesale_data['Electrical']= housesale_data['Electrical'].fillna(housesale_data['Electrical'].mode()[0])

# Columns left with null values = ['LotFrontage', 'MasVnrArea' , 'MasVnrType', 'Electrical']

In [20]:
# Check remaining columns and NULL values in it
housesale_data.isna().sum().sort_values(ascending = False).head(20)

MSSubClass      0
HalfBath        0
FireplaceQu     0
Fireplaces      0
Functional      0
TotRmsAbvGrd    0
KitchenQual     0
KitchenAbvGr    0
BedroomAbvGr    0
FullBath        0
MSZoning        0
BsmtHalfBath    0
BsmtFullBath    0
GrLivArea       0
LowQualFinSF    0
2ndFlrSF        0
1stFlrSF        0
Electrical      0
GarageType      0
GarageYrBlt     0
dtype: int64

#### All Null values are handled now

In [21]:
# Get columns and number of unique values in housesale_data
unique_value_counts_per_column = housesale_data.nunique()

unique_value_counts_per_column.sort_values()

CentralAir        2
Street            2
Utilities         2
LandSlope         3
HalfBath          3
               ... 
TotalBsmtSF     721
1stFlrSF        753
BsmtUnfSF       780
GrLivArea       861
LotArea        1073
Length: 76, dtype: int64

- There are no columns with only 1 (single) value in it so we would need all columns

## Derived Features

In [22]:
# we can calculate property age at the time of sale generally older properties have low value, age of property can be important variable
housesale_data['houseAge'] = housesale_data['YrSold'] - housesale_data['YearBuilt']
housesale_data.shape

(1460, 77)

## Managing Outliers

In [23]:
numeric_columns = housesale_data.select_dtypes(exclude=object).columns
print('Total number of Numeric columns: ' + str(len(numeric_columns)))

Total number of Numeric columns: 38


In [ ]:
# Outlier Analysis using Box Plot features to visualize outliers
plt.figure(figsize = (10, 40))

plotCounter = 1
for col in numeric_columns:
    plt.subplot(20, 2, plotCounter)
    sns.boxplot(housesale_data[col])
    plotCounter = plotCounter + 1

plt.tight_layout()
plt.show()

<b>Observation:</b>

Lot of outliers are visible in LotFrontage, LotArea, MasVnrArea, BsmtFinSF2, LowQualFinSF, GrLivArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal


In [ ]:
# Based on above observations, we can drop few columns

cols_to_drop = ['PoolArea', 'MiscVal', '3SsnPorch']
housesale_data.drop(cols_to_drop, axis = 1, inplace=True)
housesale_data.shape

In [ ]:
# Drop outliers for all numeric features using IQR method

numerical_columns = housesale_data.select_dtypes(exclude=object).columns

def drop_outlier_rows(dataset, col):
    Q1 = dataset[col].quantile(.05)
    Q3 = dataset[col].quantile(.95)
    IQR = Q3 - Q1
    lower_Range = Q1 - (1.5 * IQR)
    upper_Range = Q3 + (1.5 * IQR)
    dataset = dataset[(dataset[col] >= lower_Range) & (dataset[col] <= upper_Range)]
    return dataset

for col in numerical_columns:
    housesale_data = drop_outlier_rows(housesale_data, col)

# Data after removing rows
housesale_data.shape

#### Checking Correlated Variables

In [ ]:
# plot correlation graph

plt.figure(figsize = (25, 20))

# mask = np.triu(np.ones_like(housesale_data.corr(), dtype=np.bool))
sns.heatmap(housesale_data.corr(),  annot = True)

plt.show()

<b>Observations:</b>
    
- 1stFlrSF is related to TotalBsmtSf 
- TotRmsAbvGrd is related to GrLivArea 
- GarageYrBlt is realted to YearBlt 
- GarageArea is related to GarageCars 
- SalePrice is related to OverallQual, GrLivArea, GarageCars, GarageArea, TotalRmsAbvGrd, TotalBsmtSf, 1stFlrSf, FullBath

In [ ]:
# According to above observations, removing some of the correlated columns

col_names_related = ['TotRmsAbvGrd', 'GarageYrBlt', 'GarageArea']
housesale_data.drop(col_names_related, axis = 1, inplace=True)
housesale_data.shape

# <u>Step 2: Visualising the Data</u>

### Performing Exploratory Data Analysis

### Analysing Target Variable - SalePrice

In [ ]:
sns.distplot(housesale_data['SalePrice'])

<b>Since the Saleprice figures are skewed towards left, we will apply the log transformation to obtain a centralized data</b>

In [ ]:
#Log Transformation
housesale_data['SalePrice']=np.log1p(housesale_data['SalePrice'])

In [ ]:
sns.distplot(housesale_data['SalePrice'])

### Analysing Numerical Variables

In [ ]:
numeric_columns = housesale_data.select_dtypes(exclude=object).columns

# Plotting Scatter plots to understand feature trend with sale price
plt.figure(figsize=(10, 40))

plotCounter = 1
for col in numeric_columns:
    plt.subplot(16, 2, plotCounter)
    sns.scatterplot(x = col, y = 'SalePrice', data = housesale_data)
    plotCounter = plotCounter + 1

plt.tight_layout()
plt.show()


<b>Observations:</b>
    
- LotFrontage, LotArea, GrLivArea, GarageArea are few of the features that show a linear trend with salePrice
- KitchenAbvGr, LowQualFinSF features have single vertical bar in graph - that shows all values for these features are same, and these can be dropped

In [ ]:
# Based on above observations, few features can be dropped
col_names_with_all_values_same = ['LowQualFinSF', 'KitchenAbvGr']
housesale_data.drop(col_names_with_all_values_same, axis = 1, inplace=True)
housesale_data.shape

### Analysing Categorical Variables

In [ ]:
category_columns = housesale_data.select_dtypes(include=object).columns
print('Total Category Columns present are :' + str(len(category_columns)))

In [ ]:
# Plotting histogram plots to understand feature distribution
plt.figure(figsize=(10, 40))

plotCounter = 1
for col in category_columns:
    plt.subplot(20, 2, plotCounter)
    sns.countplot(col, data = housesale_data)
    plotCounter = plotCounter + 1

plt.tight_layout()
plt.show()

<b>Observations:</b>

Street, Utilities, Condition2, Heating, CentralAir, Functional, GarageQual are feature majorly dominated by single category

In [ ]:
# Based on above observations, few columns can be dropped
columns_to_drop_aftr_catvar_analysis = ['Street', 'Utilities', 'Condition2', 'Heating', 'CentralAir', 'Functional', 'GarageQual']
housesale_data.drop(columns_to_drop_aftr_catvar_analysis, axis = 1, inplace=True)

# Data after removing variables
housesale_data.shape

# <u>Step 3: Data Preparation for modelling</u>

## Creating dummy variables

In [ ]:
# Create Dummy variables for all categorical columns

cat_columns = housesale_data.select_dtypes(include=object).columns
cat_columns

In [ ]:
for col in cat_columns:
    datasetCol = pd.get_dummies(housesale_data[col], prefix = col, drop_first=True)
    housesale_data = pd.concat([housesale_data, datasetCol], axis = 1)
    housesale_data.drop(col, axis = 1, inplace = True)

housesale_data.head()

In [ ]:
housesale_data.shape

In [ ]:
column_names = housesale_data.columns
print("Column Names:")
for column_name in column_names:
    print(column_name)

# <u>Step 4: Splitting the Data into Training and Testing Sets</u>

In [ ]:
# Splitting data in training & test sets
y = housesale_data.pop('SalePrice')
X = housesale_data


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 100)

In [ ]:
# Printing train & test data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Rescaling required variables

In [ ]:
X_train.head()

In [ ]:
# Using MinMaxScaler to Rescaling the features/variables
scaler = MinMaxScaler()

In [ ]:
# Rescaling all numeric columns
numeric_columns = X_train.select_dtypes(exclude=object).columns

X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])

X_train.head()

In [ ]:
# Transfoming the test X_test columns, here we just transform and not fit & tranform
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

X_test.head()

# <u>Step 5: Model Building and Evaluation</u>

In [ ]:
# Common function for regression
def calculate_score_metrics(y_train, y_pred_train, y_test, y_pred_test):
    metric = []
    
    r2_train = r2_score(y_train, y_pred_train)
    print('R2 score train : ', r2_train)
    metric.append(r2_train)

    r2_test = r2_score(y_test, y_pred_test)
    print('R2 score test  : ', r2_test)
    metric.append(r2_test)

    rss_train = np.sum(np.square(y_train - y_pred_train))
    print('RSS score train : ', rss_train)
    metric.append(rss_train)

    rss_test = np.sum(np.square(y_test - y_pred_test))
    print('RSS score test  : ', rss_test)
    metric.append(rss_test)

    mse_train = mean_squared_error(y_train, y_pred_train)
    print('MSE score train : ', mse_train)
    metric.append(mse_train ** 0.5)

    mse_test = mean_squared_error(y_test, y_pred_test)
    print('MSE score test  : ', mse_test)
    metric.append(mse_test ** 0.5)
    
    return metric

In [ ]:
# Running RFE 
# Since there are more than 250 variables for analysis, we will run RFE to select some that have high predictive power
lm = LinearRegression()
lm.fit(X_train, y_train)
 # running RFE for top 100 variables
rfe = RFE(lm, n_features_to_select=100)            
rfe = rfe.fit(X_train, y_train)

In [ ]:
# Check the ranks
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
# Select the top 100 variables

col = X_train.columns[rfe.support_]
col

In [ ]:
X_train.columns[~rfe.support_]

In [ ]:
# Creating X_test dataframe with RFE selected variables
X_train_rfe = X_train[col]

In [ ]:
X_train_rfe = pd.DataFrame(X_train[col])

In [ ]:
X_train_rfe.head()

In [ ]:
X_train_rfe.shape

In [ ]:
# predict
y_pred_train = lm.predict(X_train)
y_pred_test = lm.predict(X_test)

metric_linear_regression = calculate_score_metrics(y_train, y_pred_train, y_test, y_pred_test)

## Since the Test R2 is too low, we will check for some alternate methods of Regression

In [ ]:
######################################################################

# add constant
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

# create model
lr = sm.OLS(y_train, X_train_sm)
 
#fit
lm = lr.fit()

# summary
lm.summary()

In [ ]:
# Calculate score metrics - R2 Score, RSS, RSME
y_pred_train = lm.predict(X_train_sm)
y_pred_test = lm.predict(X_test_sm)

metric_linear_regression = calculate_score_metrics(y_train, y_pred_train, y_test, y_pred_test)

In [ ]:
# Residual analysis
y_res = y_train - y_pred_train # Residuals

# Residual v/s predictions plot
plt.scatter(y_pred_train , y_res)
plt.axhline(y=0, color='r', linestyle=':')
plt.show()

In [ ]:
# Distribution of errors
p = sns.distplot(y_res,kde=True)

p = plt.title('Normality of error terms/residuals')
plt.xlabel("Residuals")
plt.show()

######################################################################

# <u>Step 6: Ridge Regression</u>

Let's now try predicting house prices using ridge regression.

In [ ]:
# list of alphas to tune - if value too high it will lead to underfitting, if it is too low, 
# it will not handle the overfitting
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}

ridge = Ridge()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error',  
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            
model_cv.fit(X_train, y_train)

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=5]
cv_results.head()

In [ ]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('int32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')
plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()


### Since the Negative Mean Absolute Error stabilises at alpha = 2, we will choose this for further analysis

In [ ]:
# Printing the best hyperparameter alpha
print(model_cv.best_params_)

In [ ]:
# Fitting Ridge model for best alpha
alpha = 2.0
ridge = Ridge(alpha=alpha)

ridge.fit(X_train, y_train)
print(ridge.coef_)

In [ ]:
# ridge model parameters
model_parameters = list(ridge.coef_)
model_parameters.insert(0, ridge.intercept_)
model_parameters = [round(x, 3) for x in model_parameters]
cols = X.columns
cols = cols.insert(0, "constant")
list(zip(cols, model_parameters))

In [ ]:
# ridge regression
lm = Ridge(alpha=2)
lm.fit(X_train, y_train)

# predict
y_pred_train = lm.predict(X_train)
y_pred_test = lm.predict(X_test)

metric_ridge_regression = calculate_score_metrics(y_train, y_pred_train, y_test, y_pred_test)

In [ ]:
# Residual analysis
y_res = y_train - y_pred_train # Residuals

# Residual v/s predictions plot
plt.scatter(y_pred_train , y_res)
plt.axhline(y=0, color='r', linestyle=':')
plt.show()

In [ ]:
# Distribution of errors
p = sns.distplot(y_res,kde=True)

p = plt.title('Normality of error terms/residuals')
plt.xlabel("Residuals")
plt.show()

In [ ]:
# Create a dataframe for Ridge Coefficients
mod_ridge = list(zip(cols, model_parameters))

In [ ]:
params_ridge = pd.DataFrame(mod_ridge)
params_ridge.columns = ['Variable', 'Coeff']
res=params_ridge.sort_values(by=['Coeff'], ascending = False)
res.head(20)

In [ ]:
# Sorting the coefficients in ascending order
params_ridge = params_ridge.sort_values((['Coeff']), axis = 0, ascending = False)
params_ridge

In [ ]:
## since there were few coefficients at 0, we removed them from features
params_ridge = pd.DataFrame(params_ridge[(params_ridge['Coeff'] != 0)])
params_ridge

In [ ]:
params_ridge.shape

# <u>Step 7: Lasso Regression</u>

In [ ]:
# Applying Lasso

# list of alphas to tune
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}
lasso = Lasso()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_cv.fit(X_train, y_train) 

In [ ]:
# cv_results
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=1]
cv_results.head()

In [ ]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('float32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')
plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()

From the above graph we can see that the Negative Mean Absolute Error is quite low at alpha = 0.4 and stabilises thereafter,

but we will choose a low value of alpha to balance the trade-off between Bias-Variance

and to get the coefficients of smallest of features.

In [ ]:
# Printing the best hyperparameter alpha
print(model_cv.best_params_)

In [ ]:
# At alpha = 0.01, even the smallest of negative coefficients that have some predictive power towards 'SalePrice' have been generated

alpha = 0.01
lasso = Lasso(alpha=alpha)

lasso.fit(X_train, y_train)
lasso.coef_

### The advantage of this technique is clearly visible here as Lasso brings the coefficients of insignificant features to zero

In [ ]:
# lasso model parameters
model_parameters = list(lasso.coef_ )
model_parameters.insert(0, lasso.intercept_)
model_parameters = [round(x, 3) for x in model_parameters]
cols = X.columns
cols = cols.insert(0, "constant")
list(zip(cols, model_parameters))

In [ ]:
# prediction on the test set(Using R2)
y_pred_train = lasso.predict(X_train)
y_pred_test = lasso.predict(X_test)

In [ ]:
# Calculate score metrics - R2 Score, RSS, RSME
metric_lasso_regression = calculate_score_metrics(y_train, y_pred_train, y_test, y_pred_test)

In [ ]:
# Residual analysis
y_res = y_train - y_pred_train # Residuals

# Residual v/s predictions plot
plt.scatter(y_pred_train , y_res)
plt.axhline(y=0, color='r', linestyle=':')
plt.show()

In [ ]:
# Distribution of errors
p = sns.distplot(y_res,kde=True)

p = plt.title('Normality of error terms/residuals')
plt.xlabel("Residuals")
plt.show()

In [ ]:
# Create a dataframe for Ridge Coefficients
mod_lasso = list(zip(cols, model_parameters))

In [ ]:
params_lasso = pd.DataFrame(mod_lasso)
params_lasso.columns = ['Variable', 'Coeff']
res=params_lasso.sort_values(by=['Coeff'], ascending = False)
res.head(20)

In [ ]:
# Sorting the coefficients in ascending order
params_lasso = params_lasso.sort_values((['Coeff']), axis = 0, ascending = False)
params_lasso

In [ ]:
## since there were few coefficients at 0, we removed them from features
params_lasso = pd.DataFrame(params_lasso[(params_lasso['Coeff'] != 0)])
params_lasso

In [ ]:
params_lasso.shape

# <u>Step 8: Model Comparison</u>

In [ ]:
# Creating a table which contain all the metrics

score_table = {'Metric': ['R2 Score (Train)','R2 Score (Test)','RSS (Train)','RSS (Test)','RMSE (Train)','RMSE (Test)']}
score_metric = pd.DataFrame(score_table)

lr_metric = pd.Series(metric_linear_regression, name = 'Linear Regression')
rg_3_metric = pd.Series(metric_ridge_regression, name = 'Ridge Regression')
ls_100_metric = pd.Series(metric_lasso_regression, name = 'Lasso Regression')

score_metric = pd.concat([score_metric, lr_metric, rg_3_metric, ls_100_metric], axis = 1)

score_metric

<b>Observation:</b>

Though the model performance by Ridge Regression was better in terms of R2 values of Train and Test, it is better to use Lasso, since it brings and assigns a zero value to insignificant features, enabling us to choose
the predictive variables.

-----

# <u>Conclusion</u>

In [ ]:
significance_matrix = pd.DataFrame()

significance_matrix['Feature Name'] = X_test.columns
significance_matrix['Coefficient'] = pd.Series(lasso.coef_)
significance_matrix['Absolute Coefficient'] = pd.Series(np.abs(lasso.coef_))

significance_matrix = significance_matrix.loc[significance_matrix['Coefficient'] != 0]

significance_matrix.sort_values(by=['Absolute Coefficient'], inplace=True, ascending=False)

significance_matrix.head(10)

Significant top 10 features that affect house price are below, and there significance is as shown by Coefficients above

---

<b>INFERENCE</b>

Surprise Housing can keep track of below predictors affecting the price of the house.

<b>Positive Coefficient Variables - suggest a high sale value</b>

- GrLivArea	: Above grade (ground) living area square feet
- OverallQual :	Rates the overall material and finish of the house
- GarageCars :	Size of garage in car capacity
- YearRemodAdd :	Remodel date (same as construction date if no remodeling or additions)
- MSZoning :	Identifies the general zoning classification of the sale

<b>Negative Coefficient Vaiables - suggest a decrease in sale value</b>

- FireplaceQu :	Fireplace quality
- ExterQual: Evaluates the quality of the material on the exterior

---